# Data Cleaning

* Formatting data into readable formats
* Removing unnecessary features

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,  confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer




In [11]:
import csv
import pandas as pd


data = pd.read_csv('Pokemon Database.csv')


for index, pokemon in data.iterrows():
    for column in data.columns:
        if isinstance(pokemon[column],str):
            data.at[index, column] = pokemon[column][1:-1] 
            

data["Alternate Form Name"] = data["Alternate Form Name"].replace({
    "Hisui": "Hisuian",
    "Alola": "Alolan",
    "Galar": "Galarian"
})    
data=data[data["Alternate Form Name"]!="Gigantamax"]
            
for index, pokemon in data.iterrows():
    if pd.isna(pokemon['Legendary Type']):
        data.at[index,"Legendary Type"] = "Regular"
    if pd.isna(pokemon["Secondary Type"]):
        data.at[index, "Secondary Type"] = pokemon["Primary Type"]     
    alternate_form = pokemon['Alternate Form Name']
    if not pd.isna(alternate_form) and isinstance(alternate_form,str):
        if alternate_form=="Mega X" or alternate_form == "Mega Y":
            data.at[index,"Pokemon Name"] = f"Mega {data.at[index,"Pokemon Name"]} {alternate_form[-1]}"
        elif pokemon["Pokemon Name"] in ["Unown", "Hoopa"]:
            data.at[index,"Pokemon Name"] = f"{data.at[index,"Pokemon Name"]} {alternate_form}"
        else:
            data.at[index,"Pokemon Name"] = f"{alternate_form} {data.at[index,"Pokemon Name"]}"
 
columns = ['Pokedex Number', 'Pokemon Name', 'Legendary Type', 'Pokemon Height', 'Pokemon Weight', 
            'Primary Type','Secondary Type', 'Male Ratio', 'Female Ratio', 'Base Happiness','Health Stat', 
            'Attack Stat', 'Defense Stat', 'Special Attack Stat', 'Special Defense Stat', 'Speed Stat', 
            'Base Stat Total', 'Health EV', 'Attack EV', 'Defense EV', 'Special Attack EV', 'Special Defense EV', 
            'Speed EV', 'EV Yield Total', 'Catch Rate', 'Experience Growth', 'Experience Growth Total','Egg Cycle Count']    
        
data = data[columns]
data.to_csv('Processed Data.csv')



In [ ]:
features = ['Legendary Type', 'Pokemon Height', 'Pokemon Weight', 'Male Ratio', 'Female Ratio', 'Base Happiness',
            'Health Stat', 'Attack Stat', 'Defense Stat', 'Special Attack Stat', 'Special Defense Stat', 'Speed Stat', 
            'Base Stat Total', 'Health EV', 'Attack EV', 'Defense EV', 'Special Attack EV', 'Special Defense EV', 
            'Speed EV', 'EV Yield Total', 'Catch Rate', 'Experience Growth', 'Experience Growth Total','Egg Cycle Count']    

categorical_features = ['Legendary Type', "Experience Growth"]  
numerical_features = [col for col in features if col not in categorical_features]

transformer = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),  
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  
])

data_transformed = transformer.fit_transform(data[features])
encoded = transformer.get_feature_names_out()
 